# Section 0: Import relevant functions

In [30]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import random
# Dependencies and Setup
# SQL Alchemy
from sqlalchemy import create_engine

In [31]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, PrimaryKeyConstraint

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
import sqlite3

# Section 1: Load, view, and begin to clean the data

Dataset #1: Electrical vehicle charging station by states location

In [32]:
engine = create_engine("sqlite:///data/db.sqlite")

meta = MetaData()
meta.drop_all(engine)

In [33]:
stations_df = pd.read_csv('data/alt_fuel_stations.csv')

In [34]:
stations_df.head()

,Fuel Type Code,Station Name,Street Address,City,State,ZIP,Station Phone,Status Code,Groups With Access Code,Access Days Time,...,Date Last Confirmed,ID,Updated At,Owner Type Code,Open Date,EV Connector Types,Country,Access Code,Facility Type,EV Pricing
0,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,24 hours daily; pay lot,...,5/9/19,1523,2020-01-21 18:34:38 UTC,P,8/30/95,J1772,US,public,PARKING_GARAGE,NaN
1,ELEC,Cherokee & Hollywood Parking Garage,1718 N Cherokee Ave,Los Angeles,CA,90028,213-550-9904,E,Public,24 hours daily; pay lot,...,5/9/19,1526,2020-01-21 18:34:38 UTC,P,8/15/95,J1772,US,public,PARKING_GARAGE,Free
2,ELEC,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,626-575-6800,E,Public,24 hours daily,...,9/9/19,1583,2020-01-21 18:34:38 UTC,SG,10/15/96,J1772,US,public,STATE_GOV,Free
3,ELEC,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,858-554-9100,E,Public,24 hours daily,...,12/12/19,6355,2020-01-21 18:34:38 UTC,P,7/30/97,J1772,US,public,HOSPITAL,Pay to Park
4,ELEC,San Diego Wild Animal Park,15500 San Pasqual Valley Rd,Escondido,CA,92027,760-747-8702,E,Public,24 hours daily,...,2/6/19,6384,2020-01-21 18:34:38 UTC,P,9/2/14,J1772,US,public,PAY_LOT,Free


In [35]:
stations_df.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address', 'City', 'State',
       'ZIP', 'Station Phone', 'Status Code', 'Groups With Access Code',
       'Access Days Time', 'Cards Accepted', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV Network', 'EV Network Web', 'Geocode Status',
       'Latitude', 'Longitude', 'Date Last Confirmed', 'ID', 'Updated At',
       'Owner Type Code', 'Open Date', 'EV Connector Types', 'Country',
       'Access Code', 'Facility Type', 'EV Pricing'],
      dtype='object')

In [36]:
# keep only wanted columns & Reorder - make ID column first
stations_df = stations_df[['ID','Station Name','Street Address','City','State','ZIP','Station Phone','Status Code',
                           'Groups With Access Code','Access Days Time','Geocode Status',
       'Latitude', 'Longitude','Open Date','EV Connector Types','Access Code', 'Facility Type', 'EV Pricing']]

In [37]:
# Rename column names
stations_df.rename(columns={'ID': 'id', 'Station Name':'station_name', 'Street Address': 'street_address',
                            'City': 'city', 'State': 'state','ZIP': 'zip','Station Phone': 'station_phone',
                            'Status Code': 'status_code', 'Groups With Access Code': 'groups_access_code', 
                            'Access Days Time': 'access_days_time', 'Geocode Status': 'geocode status',
                            'Latitude': 'latitude', 'Longitude': 'longitude','Open Date':'open_date','EV Connector Types':'ev_connector_types',
                            'Access Code':'access_code', 'Facility Type':'facility_type' , 'EV Pricing':'ev_pricing'
                            }, inplace=True)

In [38]:
stations_df.head()

,id,station_name,street_address,city,state,zip,station_phone,status_code,groups_access_code,access_days_time,geocode status,latitude,longitude,open_date,ev_connector_types,access_code,facility_type,ev_pricing
0,1523,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,24 hours daily; pay lot,200-9,34.040570,-118.268762,8/30/95,J1772,public,PARKING_GARAGE,NaN
1,1526,Cherokee & Hollywood Parking Garage,1718 N Cherokee Ave,Los Angeles,CA,90028,213-550-9904,E,Public,24 hours daily; pay lot,200-8,34.102404,-118.334758,8/15/95,J1772,public,PARKING_GARAGE,Free
2,1583,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,626-575-6800,E,Public,24 hours daily,GPS,34.068720,-118.064000,10/15/96,J1772,public,STATE_GOV,Free
3,6355,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,858-554-9100,E,Public,24 hours daily,GPS,32.899470,-117.243000,7/30/97,J1772,public,HOSPITAL,Pay to Park
4,6384,San Diego Wild Animal Park,15500 San Pasqual Valley Rd,Escondido,CA,92027,760-747-8702,E,Public,24 hours daily,200-9,33.098589,-117.004433,9/2/14,J1772,public,PAY_LOT,Free


In [39]:
# Identify incomplete rows
#stations_df.count()

In [40]:
print(stations_df.columns)

Index(['id', 'station_name', 'street_address', 'city', 'state', 'zip',
       'station_phone', 'status_code', 'groups_access_code',
       'access_days_time', 'geocode status', 'latitude', 'longitude',
       'open_date', 'ev_connector_types', 'access_code', 'facility_type',
       'ev_pricing'],
      dtype='object')


In [41]:
stations_df.head()

,id,station_name,street_address,city,state,zip,station_phone,status_code,groups_access_code,access_days_time,geocode status,latitude,longitude,open_date,ev_connector_types,access_code,facility_type,ev_pricing
0,1523,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,24 hours daily; pay lot,200-9,34.040570,-118.268762,8/30/95,J1772,public,PARKING_GARAGE,NaN
1,1526,Cherokee & Hollywood Parking Garage,1718 N Cherokee Ave,Los Angeles,CA,90028,213-550-9904,E,Public,24 hours daily; pay lot,200-8,34.102404,-118.334758,8/15/95,J1772,public,PARKING_GARAGE,Free
2,1583,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,626-575-6800,E,Public,24 hours daily,GPS,34.068720,-118.064000,10/15/96,J1772,public,STATE_GOV,Free
3,6355,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,858-554-9100,E,Public,24 hours daily,GPS,32.899470,-117.243000,7/30/97,J1772,public,HOSPITAL,Pay to Park
4,6384,San Diego Wild Animal Park,15500 San Pasqual Valley Rd,Escondido,CA,92027,760-747-8702,E,Public,24 hours daily,200-9,33.098589,-117.004433,9/2/14,J1772,public,PAY_LOT,Free


In [42]:
stations_df.shape

(27346, 18)

In [43]:
# Return non-NAN rows in each column
#print(stations_df.count())

In [44]:
# Return total number of rows including NAN in the dataframe
stations_df.size

492228

In [45]:
#stations_df.describe()

Dataset #2: Pandas Web Scraping: How average temperatures in different US regions have increased with time

In [46]:
url = 'https://science2017.globalchange.gov/chapter/6/'
table = pd.read_html(url)
temp_df = table[2]
temp_df

,NCA Region,Change in Annual Average Temperature,Change in Annual Average Maximum Temperature,Change in Annual Average Minimum Temperature
0,Contiguous U.S.,1.23°F,1.06°F,1.41°F
1,Northeast,1.43°F,1.16°F,1.70°F
2,Southeast,0.46°F,0.16°F,0.76°F
3,Midwest,1.26°F,0.77°F,1.75°F
4,Great Plains North,1.69°F,1.66°F,1.72°F
5,Great Plains South,0.76°F,0.56°F,0.96°F
6,Southwest,1.61°F,1.61°F,1.61°F
7,Northwest,1.54°F,1.52°F,1.56°F
8,Alaska,1.67°F,1.43°F,1.91°F
9,Hawaii,1.26°F,1.01°F,1.49°F


Average Temperatures from 1986-2016 vs. Average Temperatures from 1901-1960 in different US Regions:

Relative to temperatures early in 20th Century, all US regions have experienced temperature increases. 
This is true for average, minimum avg, and maximum avg temperatures. 
The greatest temperature increases have taken place in the Western USA (SW/Hawaii, NW/Alaska).

In [47]:
# Convert tabular data to html if needed for the index.html site
temp_table = temp_df.to_html()

Dataset #3: Obtain last 30 years of US National CO2 Emissions Data

Data obtained from Kaggle Dataset: https://www.kaggle.com/yoannboyere/co2-ghg-emissionsdata

In [48]:
co2_file = "data/co2_emission.csv"
co2_file_df = pd.read_csv(co2_file)

US_co2 = co2_file_df.loc[co2_file_df["Entity"] == "United States"]
US_co2_df = US_co2.tail(30)
US_co2_df

,Entity,Code,Year,Annual CO₂ emissions (tonnes )
19898,United States,USA,1988,4.885591e+09
19899,United States,USA,1989,4.948021e+09
19900,United States,USA,1990,5.121264e+09
19901,United States,USA,1991,5.070839e+09
19902,United States,USA,1992,5.174059e+09
19903,United States,USA,1993,5.284688e+09
19904,United States,USA,1994,5.377987e+09
19905,United States,USA,1995,5.439213e+09
19906,United States,USA,1996,5.626187e+09
19907,United States,USA,1997,5.703891e+09


Dataset #4: Obtain last 30 years of US State by State Emissions Data

Data obtained from Energy Information Administration Dataset: https://www.eia.gov/environment/emissions/state/

In [49]:
states_co2 = "data/state_emissions.csv"
states_co2 = pd.read_csv(states_co2, error_bad_lines=False)

# Rename columns 
states_co2 = states_co2.rename(columns={"Unnamed: 1":"Year_1990", "Unnamed: 2":"Year_1991", "Unnamed: 3":"Year_1992",
                                       "Unnamed: 4":"Year_1993", "Unnamed: 5":"Year_1994", "Unnamed: 6":"Year_1995",
                                       "Unnamed: 7":"Year_1996","Unnamed: 8":"Year_1997", "Unnamed: 9":"Year_1998",
                                       "Unnamed: 10":"Year_1999", "Unnamed: 11":"Year_2000", "Unnamed: 12":"Year_2001",
                                       "Unnamed: 13":"Year_2002", "Unnamed: 14":"Year_2003", "Unnamed: 15":"Year_2004",
                                       "Unnamed: 16":"Year_2005", "Unnamed: 17":"Year_2006", "Unnamed: 18":"Year_2007",
                                       "Unnamed: 19":"Year_2008", "Unnamed: 20":"Year_2009", "Unnamed: 21":"2010",
                                       "Unnamed: 22":"Year_2011","Unnamed: 23":"Year_2012", "Unnamed: 24":"Year_2013",
                                       "Unnamed: 25":"Year_2014","Unnamed: 26":"Year_2015", "Unnamed: 27":"Year_2016",
                                       "Unnamed: 28":"Year_2017"})


states_co2_df = states_co2
states_co2_df

,State,Year_1990,Year_1991,Year_1992,Year_1993,Year_1994,Year_1995,Year_1996,Year_1997,Year_1998,...,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Percent,Absolute,Percent.1,Absolute.1
0,AL,110.0,114.4,121.0,125.6,123.6,131.3,137.3,134.1,133.4,...,123.0,120.7,123.1,120.0,114.9,109.0,-0.90%,-1.0,-5.10%,-5.9
1,AK,34.5,35.2,36.6,36.4,36.3,40.9,41.8,41.8,42.9,...,38.0,36.0,35.2,36.1,34.7,34.3,-0.70%,-0.2,-1.30%,-0.4
2,AZ,62.9,63.8,66.6,68.9,71.6,66.5,68.3,71.4,76.3,...,91.3,95.1,93.1,90.9,87.0,86.1,37.00%,23.3,-1.00%,-0.8
3,AR,50.9,49.8,51.5,50.5,54.4,57.8,60.4,59.3,60.7,...,65.9,68.4,69.0,59.2,62.3,64.3,26.20%,13.4,3.10%,2.0
4,CA,357.6,345.3,349.5,338.9,354.9,343.8,342.4,344.7,355.1,...,351.5,354.2,351.5,359.3,359.3,360.9,0.90%,3.3,0.50%,1.7
5,CO,66.0,67.3,68.3,72.0,72.4,72.5,75.3,75.4,77.7,...,90.6,91.2,91.8,90.5,87.2,87.6,32.70%,21.6,0.50%,0.4
6,CT,40.7,39.9,40.2,38.2,37.5,36.8,39.6,42.6,40.2,...,33.9,34.7,35.0,36.3,34.1,33.6,-17.50%,-7.1,-1.50%,-0.5
7,DE,16.7,17.5,16.3,17.7,17.1,16.1,16.7,15.3,14.9,...,13.3,13.1,12.7,12.8,13.2,12.3,-26.00%,-4.3,-6.80%,-0.9
8,DC,4.5,4.4,4.4,4.5,4.5,4.4,4.4,4.3,4.1,...,2.7,2.8,3.0,3.0,2.8,2.6,-40.90%,-1.8,-5.10%,-0.1
9,FL,188.3,189.5,191.9,194.9,202.3,207.5,213.1,218.1,231.0,...,222.1,221.1,227.0,231.4,229.2,228.0,21.00%,39.6,-0.50%,-1.3


# Section 4: Load the data into the database

In [51]:
# Create connection to engine
conn = engine.connect()

In [52]:
#engine.execute("SELECT * FROM Stations").fetchall()

In [53]:
# write records stored in a DataFrame to a SQL database.
stations_data = stations_df.to_sql('Stations', con=engine, if_exists='replace')

# Section 5: Validate and query the data

In [54]:
# Query all objects into the database

pd.read_sql_query('select * from Stations', con=engine).head()

,index,id,station_name,street_address,city,state,zip,station_phone,status_code,groups_access_code,access_days_time,geocode status,latitude,longitude,open_date,ev_connector_types,access_code,facility_type,ev_pricing
0,0,1523,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,24 hours daily; pay lot,200-9,34.040570,-118.268762,8/30/95,J1772,public,PARKING_GARAGE,None
1,1,1526,Cherokee & Hollywood Parking Garage,1718 N Cherokee Ave,Los Angeles,CA,90028,213-550-9904,E,Public,24 hours daily; pay lot,200-8,34.102404,-118.334758,8/15/95,J1772,public,PARKING_GARAGE,Free
2,2,1583,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,626-575-6800,E,Public,24 hours daily,GPS,34.068720,-118.064000,10/15/96,J1772,public,STATE_GOV,Free
3,3,6355,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,858-554-9100,E,Public,24 hours daily,GPS,32.899470,-117.243000,7/30/97,J1772,public,HOSPITAL,Pay to Park
4,4,6384,San Diego Wild Animal Park,15500 San Pasqual Valley Rd,Escondido,CA,92027,760-747-8702,E,Public,24 hours daily,200-9,33.098589,-117.004433,9/2/14,J1772,public,PAY_LOT,Free


In [55]:
engine.execute("SELECT * FROM Stations").fetchall()

[(0, 1523, 'Los Angeles Convention Center', '1201 S Figueroa St', 'Los Angeles', 'CA', '90015', '213-741-1151', 'E', 'Public', '24 hours daily; pay lot', '200-9', 34.04057, -118.26876200000001, '8/30/95', 'J1772', 'public', 'PARKING_GARAGE', None),
 (1, 1526, 'Cherokee & Hollywood Parking Garage', '1718 N Cherokee Ave', 'Los Angeles', 'CA', '90028', '213-550-9904', 'E', 'Public', '24 hours daily; pay lot', '200-8', 34.102404, -118.33475800000001, '8/15/95', 'J1772', 'public', 'PARKING_GARAGE', 'Free'),
 (2, 1583, 'California Air Resources Board', '9530 Telstar Ave', 'El Monte', 'CA', '91731', '626-575-6800', 'E', 'Public', '24 hours daily', 'GPS', 34.06872, -118.064, '10/15/96', 'J1772', 'public', 'STATE_GOV', 'Free'),
 (3, 6355, 'Scripps Green Hospital', '10666 N Torrey Pines Rd', 'La Jolla', 'CA', '92037', '858-554-9100', 'E', 'Public', '24 hours daily', 'GPS', 32.89947, -117.243, '7/30/97', 'J1772', 'public', 'HOSPITAL', 'Pay to Park'),
 (4, 6384, 'San Diego Wild Animal Park', '1550

In [58]:
# Save US Emissions Database into SQLite Database

from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/db.sqlite', echo=True)
sqlite_connection = engine.connect()

sqlite_table = "US_co2"
US_co2_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2020-10-15 14:03:24,357 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-15 14:03:24,357 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:03:24,358 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-15 14:03:24,359 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:03:24,361 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("US_co2")
2020-10-15 14:03:24,362 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:03:24,363 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("US_co2")
2020-10-15 14:03:24,363 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:03:24,365 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "US_co2" (
	"index" BIGINT, 
	"Entity" TEXT, 
	"Code" TEXT, 
	"Year" BIGINT, 
	"Annual CO₂ emissions (tonnes )" FLOAT
)


2020-10-15 14:03:24,365 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:03:24,381 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10

In [62]:
# Save State Emissions Database into SQLite Database

from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/db.sqlite', echo=True)
sqlite_connection = engine.connect()

sqlite_table2 = "States_co2"
states_co2_df.to_sql(sqlite_table2, sqlite_connection, if_exists='replace')

2020-10-15 14:05:34,236 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-15 14:05:34,236 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:05:34,237 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-15 14:05:34,238 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:05:34,245 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("States_co2")
2020-10-15 14:05:34,245 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:05:34,246 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("States_co2")
2020-10-15 14:05:34,247 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 14:05:34,254 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "States_co2" (
	"index" BIGINT, 
	"State" TEXT, 
	"Year_1990" FLOAT, 
	"Year_1991" FLOAT, 
	"Year_1992" FLOAT, 
	"Year_1993" FLOAT, 
	"Year_1994" FLOAT, 
	"Year_1995" FLOAT, 
	"Year_1996" FLOAT, 
	"Year_1997" FLOAT, 
	"Year_1998" FLOAT, 
	"Year_1999